# 학교 안전사고 현황 분석

- 코드 작성자 : 이유정
- 코드 작성일 : 2024-07-06

맡은 부분 알아서 잘 정리해주세요!!🥰

## 0. 라이브러리 및 데이터 임포트

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
df = pd.read_csv('../../../school-accidents/code/dashboard/component/data/학교안전사고데이터통합/학교안전사고데이터_5개년통합_월계절추가.csv')

df['사고발생일'] = pd.to_datetime(df['사고발생일'])
df['연도'] = df['사고발생일'].map(lambda x : x.year)

df_2019 = df[df['사고발생일'].between('2019-01-01', '2019-12-31')]
df_2020 = df[df['사고발생일'].between('2020-01-01', '2020-12-31')]
df_2021 = df[df['사고발생일'].between('2021-01-01', '2021-12-31')]
df_2022 = df[df['사고발생일'].between('2022-01-01', '2022-12-31')]
df_2023 = df[df['사고발생일'].between('2023-01-01', '2023-12-31')]

## 1. 학교 안전사고 현황 EDA

### 1-1. 연도별 사고 현황

### 1-2. 지역별 사고 현황

- 시도별 사고 현황
    - 시도별 총 사고건수
    - 시도별 연도별 전체 사고건수
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 학교 비율
        -  전체 학교 중 안전사고가 발생한 학교의 비율(2021년 동작구의 전체 학교 N곳 중 M곳의 학교에서 안전사고가 발생했다.)
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 건수
- 시군구별 사고 현황
    - 시군구별 총 사고 건수 (시도별에서 시도를 선택하면 확인할 수 있도록)
    - 시군구별 연도별 총 사고건수
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 학교 비율
        -  전체 학교 중 안전사고가 발생한 학교의 비율(2021년 동작구의 전체 학교 N곳 중 M곳의 학교에서 안전사고가 발생했다.)
    - 시도별 연도별 전체 학교 수 대비 안전사고 발생 건수

In [5]:
df_2023.교육청.unique()

'''
교육청(시도)
교육지원청(시군구) --> 시군구별 분석 가능
    교육지원청은 시·도 교육청의 하급 교육행정기관으로, 교육장(3급 또는 4급 상당)의 감독 아래 시·도의 교육·학예에 관한 사무를 일부 분장한다. 
'''

'\n교육청(시도)\n교육지원청(시군구) --> 시군구별 분석 가능\n    교육지원청은 시·도 교육청의 하급 교육행정기관으로, 교육장(3급 또는 4급 상당)의 감독 아래 시·도의 교육·학예에 관한 사무를 일부 분장한다. \n'

### 1-3. 학교급별 사고 현황

- 학교급별 하루 평균 사고 수
- 학교급별 총 사고 수
- 학교급별 성별별 사고 수
- 학교급별 학년별 사고 수
- 학교급별 사고 내용(사고 시간, 사고 장소, 사고 부위, 사고 형태, 사고 당시 활동, 사고 매개물)

##### 1-3-0. 데이터 일부 추출 및 공통 함수 지정

In [6]:
# 기본 정보 확인
print(df.columns)
print(df.학교급.unique())
df.groupby('학교급').count()['구분']

Index(['구분', '학교급', '지역', '교육청', '설립유형', '사고자구분', '사고자성별', '사고자학년', '사고발생일',
       '사고발생요일', '사고발생시각', '사고시간', '사고장소', '사고부위', '사고형태', '사고당시활동', '사고매개물',
       '연도', '사고월', '계절'],
      dtype='object')
['중학교' '초등학교' '고등학교' '유치원' '특수학교' '기타학교']


학교급
고등학교    130050
기타학교      1954
유치원      42403
중학교     216165
초등학교    218830
특수학교      2558
Name: 구분, dtype: int64

In [7]:
# 사고자구분 중 '일반학생', '특수학교(학급)학생', '체육특기학생' 값만 추출
print('추출 전:', df.사고자구분.unique())

sch_df = df[df['사고자구분'].isin(['일반학생', '특수학교(학급)학생', '체육특기학생'])]
print('추출 후:', sch_df.사고자구분.unique())

# 연도 type str로 변경
sch_df['연도'] = sch_df['연도'].astype(str)

추출 전: ['일반학생' '특수학교(학급)학생' '체육특기학생' '교직원' '교육활동참여자' '교직원(계약직)' '교직원(일반직)' nan
 '교직원(교원)' '일반인' '기타(일반인 등)']
추출 후: ['일반학생' '특수학교(학급)학생' '체육특기학생']


C:\Users\cabby\AppData\Local\Temp\ipykernel_156\750086091.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sch_df['연도'] = sch_df['연도'].astype(str)


In [8]:
# 초등학교 저학년/고학년 구분        
def update_school_level(row):
    if row['학교급'] == '초등학교':
        if row['사고자학년'] == '1학년' or row['사고자학년'] == '2학년' or row['사고자학년'] == '3학년':
            return '초등학교_저학년'
        elif row['사고자학년'] == '4학년' or row['사고자학년'] == '5학년' or row['사고자학년'] == '6학년':
            return '초등학교_고학년'
        
    return row['학교급']

In [9]:
# 학교급별 총 사고 수 시각화(1)
def create_h_barchart(df, year, x_col, y_col, y_order):
    year_df = df[df['연도'] == year]
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=year_df[y_col],
        x=year_df[x_col],
        orientation='h',
        marker=dict(color='#4a5fa8'),
        hovertemplate=f'<b>%{{y}}</b><br>사고수: %{{x:,}}건<extra></extra>'
        ))
    fig.update_layout(
        font=dict(
            family='KoPubWorld돋움체_Pro',
            ),
        hoverlabel=dict(
            font_size=14,
            font_family="KoPubWorld돋움체_Pro"
            ),
        yaxis=dict(
            categoryorder='array', 
            categoryarray=y_order
            ),
        height=400,
        width=800,
        margin=dict(t=50)
        )

    return fig

In [10]:
# 학교급별 총 사고 수 시각화(2)
def create_piechart(df, year, name_col, value_col, name_order, colors):
    year_df = df[df['연도'] == year]
    
    fig = px.pie(
        year_df,
        names=name_col,
        values=value_col,
        category_orders={name_col: name_order},
        color=name_col,
        color_discrete_map=colors
        )
    fig.update_traces(texttemplate='%{label}<br><span style="font-size:15px">%{percent:.2%}</span>',
                      hovertemplate='<b>%{label}</b><br>사고수: %{value}건<br>')
    fig.update_layout(
        font=dict(
            family='KoPubWorld돋움체_Pro',
            ),
        hoverlabel=dict(
            font_size=14,
            font_family="KoPubWorld돋움체_Pro"
            ),
        showlegend=False,
        height=400,
        width=400,
        margin=dict(t=120, r=60)
        )
    
    return fig

In [11]:
# 학교급별 사고자 성별 시각화
def create_pyramid_chart(df, year, x_col, y_col):
    year_df = df[df['연도'] == year]
    male_df = year_df[year_df['사고자성별'] == '남']
    female_df = year_df[year_df['사고자성별'] == '여']

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=-male_df[x_col],
        y=male_df[y_col],
        orientation='h',
        name='남',
        customdata=male_df[[x_col, '사고수 비율']],
        marker_color='#c7e4ff',
        text=male_df['사고수 비율'].apply(lambda x: f"{x:.2f}%"),
        textposition='inside',
        texttemplate='%{text}'
        ))
    fig.add_trace(go.Bar(
        x=female_df[x_col],
        y=female_df[y_col],
        orientation='h',
        name='여',
        customdata=female_df[[x_col, '사고수 비율']],
        marker_color='#FFE0B2',
        text=female_df['사고수 비율'].apply(lambda x: f"{x:.2f}%"),
        textposition='inside',
        texttemplate='%{text}'
        ))
    fig.update_layout(
        barmode='relative',
        bargap=0.01,
        legend_orientation='h',
        legend_x=1,
        legend_y=1.24,
        legend_xanchor='right',
        legend_yanchor='top',
        font=dict(
            family='KoPubWorld돋움체_Pro',
            color='black'
            ),
        hoverlabel=dict(
            font_size=14,
            font_family="KoPubWorld돋움체_Pro"
            ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        legend_title_text=' ',
        height=430,
        width=800,
        margin=dict(t=85, r=30)
        )

    hover_text_male = '<b>%{y}</b><br>사고수: %{customdata[0]:.f}건<br>퍼센트: %{customdata[1]:.2f}%'
    hover_text_female = '<b>%{y}</b><br>사고수: %{customdata[0]:.f}건<br>퍼센트: %{customdata[1]:.2f}%'

    fig.update_traces(
        hovertemplate=hover_text_male,
        selector=dict(name='남')
        )
    fig.update_traces(
        hovertemplate=hover_text_female,
        selector=dict(name='여')
        )
    fig.update_xaxes(
        showticklabels=True,
        title_text="",
        title_font=dict(size=12)  
        )
    fig.update_yaxes(
        tickfont=dict(size=12)  
        )

    return fig


In [12]:
# 학교급별 사고자 학년 시각화
def create_donut_chart(df, year, sch_level, name_col, value_col, name_order, colors):
    sch_year_df = df[(df['연도'] == year) & (df['학교급'] == sch_level)]
    
    fig = px.pie(
        sch_year_df,
        names=name_col,
        values=value_col,
        hole=0.4,
        category_orders={name_col: name_order},
        color=name_col,
        color_discrete_map=colors,
        )
    fig.update_layout(
        annotations=[
            dict(
                text=sch_level,
                x=0.5,
                y=0.5,
                font_size=15,
                showarrow=False
                )
            ]
        )
    fig.update_traces(texttemplate='%{label}<br><span style="font-size:13px">%{percent:.2%}</span>',
                      hovertemplate='<b>%{label}</b><br>사고수: %{value:,}건<br>')
    fig.update_layout(
        font=dict(
            family='KoPubWorld돋움체_Pro',
            color='black'
            ),
        hoverlabel=dict(
            font_size=14,
            font_family="KoPubWorld돋움체_Pro"
            ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=False,
        height=360,
        width=360,
        margin=dict(t=60)
        )

    return fig

In [13]:
# 학교급별 사고 수 월별 비교 시각화
def create_stacked_barchart(df, school_type_categories, palette, months):
    fig = go.Figure()
    for category, color in zip(school_type_categories, palette):
        fig.add_trace(go.Bar(
            x=df.index,
            y=df['퍼센트'][category],
            name=category,
            marker_color=color,
            customdata=df['구분'][category],
            hovertemplate='<b>%{x}</b><br>' +
                          '학교급: ' + category + '<br>' +
                          '사고수: %{customdata}건<br>' +
                          '퍼센트: %{y}<extra></extra>'
                          ))
    fig.update_layout(
        barmode='stack',
        height=500,
        width=1100,
        font=dict(
            family='KoPubWorld돋움체_Pro',
            color='black'
            ),
        hoverlabel=dict(
            font_size=14,
            font_family="KoPubWorld돋움체_Pro"
            ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        yaxis=dict(
            ticksuffix="%", title=' '
            ),
        legend=dict(
            title_text=' '
            ),
        xaxis=dict(
            title=' ',
            title_font_family='KoPubWorld돋움체 Medium',
            tickfont=dict(size=17),
            categoryorder='array',
            categoryarray=months
            ),
        margin=dict(t=30, l=105, r=105)
        )

    return fig

In [60]:
def create_sub_barchart(df, category_col, value_col, colors):
    df = df[df['학교급'].notna()]
    unique_school_levels = df['학교급'].unique()
    
    num_rows = 2
    num_cols = 3
    fig = make_subplots(
        rows=num_rows, cols=num_cols,  
        subplot_titles=unique_school_levels,  
        horizontal_spacing=0.15,
        vertical_spacing=0.3
        )
    for i, school_level in enumerate(unique_school_levels):
        row = i // num_cols + 1
        col = i % num_cols + 1
        level_data = df[df['학교급'] == school_level].sort_values(value_col, ascending=False).head(5)
        fig.add_trace(
            go.Bar(
                y=level_data[category_col],
                x=level_data[value_col],
                name=school_level,
                orientation='h',
                marker=dict(color=colors.get(school_level)),  
                hovertemplate='<b>%{y}</b><br>사고수: %{x:,}건<extra></extra>',   
                width=0.8
                ),
            row=row, col=col
            )
    fig.update_layout(
        barmode='stack',
        font=dict(
            family='KoPubWorld Dotum',
            color='black',
            size=20
            ),
        hoverlabel=dict(
            font_size=14,
            font_family="KoPubWorld Dotum"
            ),
        height=600,  
        width=1200,  
        showlegend=False,
        margin=dict(t=100)
        )
    fig.update_yaxes(
        autorange="reversed",
        ticksuffix='  ',
        title_font_family='KoPubWorld Dotum',
        tickfont=dict(size=14)
        )   
    fig.update_xaxes(
        title=' ',
        title_font_family='KoPubWorld Dotum',
        tickfont=dict(size=14)
        )
    
    return fig

#### 1-3-1. 학교급별 하루 평균 사고 수

In [15]:
# sch_aver_acci 데이터프레임
# 연도, 학교급 기준으로 groupby 후 각 연도와 학교급별 총 사고발생일 수 계산
sch_aver_acci = sch_df.groupby(['연도', '학교급']).agg(총사고수=('사고발생일', 'count')).reset_index()
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_aver_acci['학교급'] = pd.Categorical(sch_aver_acci['학교급'], categories=custom_order, ordered=True)
sch_aver_acci = sch_aver_acci.sort_values(['연도', '학교급']).reset_index(drop=True)
# 하루 평균 사고 발생 수 계산
sch_aver_acci['하루평균사고수'] = round(sch_aver_acci['총사고수'] / 365, 2)
# 전년 대비 증감률 계산
sch_aver_acci['전년대비증감률'] = sch_aver_acci.groupby('학교급')['하루평균사고수'].pct_change().fillna(0) * 100
sch_aver_acci

C:\Users\cabby\AppData\Local\Temp\ipykernel_156\603056736.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sch_aver_acci['전년대비증감률'] = sch_aver_acci.groupby('학교급')['하루평균사고수'].pct_change().fillna(0) * 100


,연도,학교급,총사고수,하루평균사고수,전년대비증감률
0,2019,유치원,8735,23.93,0.000000
1,2019,초등학교,53728,147.20,0.000000
2,2019,중학교,43579,119.39,0.000000
3,2019,고등학교,31601,86.58,0.000000
4,2019,특수학교,545,1.49,0.000000
5,2019,기타학교,357,0.98,0.000000
6,2020,유치원,6026,16.51,-31.007104
7,2020,초등학교,10928,29.94,-79.660326
8,2020,중학교,11214,30.72,-74.269202
9,2020,고등학교,10752,29.46,-65.973666


#### 1-3-2. 학교급별 총 사고 수

In [16]:
# sch_tot_acci 데이터프레임(초등학교 저학년/고학년 구분X)
# 연도, 학교급 기준으로 groupby
sch_tot_acci = sch_df.groupby(['연도', '학교급']).size().reset_index(name='총 사고수')
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교", "기타학교"]
sch_tot_acci['학교급'] = pd.Categorical(sch_tot_acci['학교급'], categories=custom_order, ordered=True)
sch_tot_acci = sch_tot_acci.sort_values(['연도', '학교급']).reset_index(drop=True)
sch_tot_acci

,연도,학교급,총 사고수
0,2019,유치원,8735
1,2019,초등학교,53728
2,2019,중학교,43579
3,2019,고등학교,31601
4,2019,특수학교,545
5,2019,기타학교,357
6,2020,유치원,6026
7,2020,초등학교,10928
8,2020,중학교,11214
9,2020,고등학교,10752


In [17]:
# sch_tot_acci 그래프 시각화(2019년)
custom_order = ["기타학교", "특수학교", "고등학교", "중학교", "초등학교", "유치원"]
create_h_barchart(sch_tot_acci, '2019', "총 사고수", "학교급", custom_order)

In [18]:
# sch_tot_acci2 데이터프레임(초등학교 저학년/고학년 구분O)
sch_tot_acci2 = sch_df.copy()
# 초등학교 저학년/고학년 구분
sch_tot_acci2.loc[(sch_tot_acci2['학교급']=='초등학교')&(sch_tot_acci2['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_tot_acci2.loc[(sch_tot_acci2['학교급']=='초등학교')&(sch_tot_acci2['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 연도, 학교급 기준으로 groupby
sch_tot_acci2 = sch_tot_acci2.groupby(['연도', '학교급']).size().reset_index(name='총 사고수')
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교", "특수학교", "기타학교"]
sch_tot_acci2['학교급'] = pd.Categorical(sch_tot_acci2['학교급'], categories=custom_order, ordered=True)
sch_tot_acci2 = sch_tot_acci2.sort_values(['연도', '학교급']).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_tot_acci2 = sch_tot_acci2.dropna()
sch_tot_acci2 

,연도,학교급,총 사고수
0,2019,유치원,8735
1,2019,초등학교(저학년),21340
2,2019,초등학교(고학년),32195
3,2019,중학교,43579
4,2019,고등학교,31601
5,2019,특수학교,545
6,2019,기타학교,357
8,2020,유치원,6026
9,2020,초등학교(저학년),4674
10,2020,초등학교(고학년),6242


In [19]:
# 시각화를 위한 설정
custom_order = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교", "특수학교", "기타학교"]
colors = {
    "유치원": '#5c7dd2', 
    "초등학교(저학년)": '#92b8ff', 
    "초등학교(고학년)": '#aeceff', 
    "중학교": '#c7e4ff', 
    "고등학교": '#c3b7eb', 
    "특수학교": '#9590e6', 
    "기타학교": '#837ed5'
    }

In [20]:
# sch_tot_acci2 그래프 시각화(2019년)
create_piechart(sch_tot_acci2, '2019', '학교급', '총 사고수', custom_order, colors)

In [21]:
# sch_tot_acci2 그래프 시각화(2020년)
create_piechart(sch_tot_acci2, '2020', '학교급', '총 사고수', custom_order, colors)

In [22]:
# sch_tot_acci2 그래프 시각화(2021년)
create_piechart(sch_tot_acci2, '2021', '학교급', '총 사고수', custom_order, colors)

In [23]:
# sch_tot_acci2 그래프 시각화(2022년)
create_piechart(sch_tot_acci2, '2022', '학교급', '총 사고수', custom_order, colors)

In [24]:
# sch_tot_acci2 그래프 시각화(2023년)
create_piechart(sch_tot_acci2, '2023', '학교급', '총 사고수', custom_order, colors)

#### 1-3-3. 학교급별 성별별 사고 수

In [25]:
# sch_gender_acci 데이터프레임
# 기타학교 값 제외
sch_gender_acci = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_gender_acci.loc[(sch_gender_acci['학교급']=='초등학교')&(sch_gender_acci['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_gender_acci.loc[(sch_gender_acci['학교급']=='초등학교')&(sch_gender_acci['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 연도, 학교급, 사고자성별 기준으로 groupby
sch_gender_acci = sch_gender_acci.groupby(['연도', '학교급', '사고자성별']).size().reset_index(name='사고수')
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교", "특수학교", "기타학교"]
sch_gender_acci['학교급'] = pd.Categorical(sch_gender_acci['학교급'], categories=custom_order, ordered=True)
sch_gender_acci = sch_gender_acci.sort_values(['학교급','연도']).reset_index(drop=True)
# 사고자성별별 사고수 비율 계산
sch_gender_acci['사고수 비율'] = sch_gender_acci.groupby(['연도', '학교급'])['사고수'].transform(lambda x: (x / x.sum() * 100).round(2))
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_gender_acci = sch_gender_acci.dropna()
sch_gender_acci

C:\Users\cabby\AppData\Local\Temp\ipykernel_156\3710615621.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,연도,학교급,사고자성별,사고수,사고수 비율
0,2019,유치원,남,5269,60.32
1,2019,유치원,여,3466,39.68
2,2020,유치원,남,3600,59.74
3,2020,유치원,여,2426,40.26
4,2021,유치원,남,5557,60.49
5,2021,유치원,여,3630,39.51
6,2022,유치원,남,5486,60.26
7,2022,유치원,여,3618,39.74
8,2023,유치원,남,5312,59.01
9,2023,유치원,여,3690,40.99


In [26]:
# sch_gender_acci 그래프 시각화(2019년)
create_pyramid_chart(sch_gender_acci, '2019', '사고수', '학교급')

#### 1-3-4. 학교급별 학년별 사고 수

In [27]:
# sch_grade_acci 데이터프레임
# '학교급'의 '유치원', '기타학교' 값 '사고자학년'의 'nan' 값 제외
sch_grade_acci = sch_df[
    (sch_df['학교급'] != '기타학교') &
    (sch_df['학교급'] != '유치원') &
    (sch_df['사고자학년'] != 'nan')
    ]
# 연도, 학교급, 사고자학년 기준으로 groupby
sch_grade_acci = sch_grade_acci.groupby(['연도', '학교급', '사고자학년']).size().reset_index(name='사고수')
# '초등학교'의 '사고자학년'에서 '유아' 값을 제외
sch_grade_acci = sch_grade_acci[~((sch_grade_acci['학교급'] == '초등학교') & 
                                  (sch_grade_acci['사고자학년'] == '유아'))]
# '중학교'의 '사고자학년'에서 '4학년'과 '5학년' 값을 제외
sch_grade_acci = sch_grade_acci[~((sch_grade_acci['학교급'] == '중학교') & 
                                  ((sch_grade_acci['사고자학년'] == '4학년') | 
                                   (sch_grade_acci['사고자학년'] == '5학년')))]
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교", "중학교", "고등학교", "특수학교"]
sch_grade_acci['학교급'] = pd.Categorical(sch_grade_acci['학교급'], categories=custom_order, ordered=True)
sch_grade_acci = sch_grade_acci.sort_values(['학교급','연도']).reset_index(drop=True)
sch_grade_acci

,연도,학교급,사고자학년,사고수
0,2019,초등학교,1학년,7014
1,2019,초등학교,2학년,6578
2,2019,초등학교,3학년,7748
3,2019,초등학교,4학년,8669
4,2019,초등학교,5학년,10837
...,...,...,...,...
89,2023,특수학교,3학년,124
90,2023,특수학교,4학년,40
91,2023,특수학교,5학년,33
92,2023,특수학교,6학년,44


In [28]:
# 시각화를 위한 설정
# 사고자학년 순서 지정
orders = ["유아", "1학년", "2학년", "3학년", "4학년", "5학년", "6학년"]
# 그래프 색 지정
colors = {
    '유아': '#5c7dd2',
    '1학년': '#92b8ff',
    '2학년': '#aeceff',
    '3학년': '#c7e4ff',
    '4학년': '#c3b7eb',
    '5학년': '#9590e6',
    '6학년': '#837ed5'
    }

##### (1) 초등학교

In [29]:
# sch_grade_acci 그래프 시각화(2019년, 초등)
create_donut_chart(sch_grade_acci, '2019', '초등학교', '사고자학년', '사고수', orders, colors)

In [30]:
# sch_grade_acci 그래프 시각화(2020년, 초등)
create_donut_chart(sch_grade_acci, '2020', '초등학교', '사고자학년', '사고수', orders, colors)

In [31]:
# sch_grade_acci 그래프 시각화(2021년, 초등)
create_donut_chart(sch_grade_acci, '2021', '초등학교', '사고자학년', '사고수', orders, colors)

In [32]:
# sch_grade_acci 그래프 시각화(2022년, 초등)
create_donut_chart(sch_grade_acci, '2022', '초등학교', '사고자학년', '사고수', orders, colors)

In [33]:
# sch_grade_acci 그래프 시각화(2023년, 초등)
create_donut_chart(sch_grade_acci, '2023', '초등학교', '사고자학년', '사고수', orders, colors)

##### (2) 중학교

In [34]:
# sch_grade_acci 그래프 시각화(2019년, 중등)
create_donut_chart(sch_grade_acci, '2019', '중학교', '사고자학년', '사고수', orders, colors)

In [35]:
# sch_grade_acci 그래프 시각화(2020년, 중등)
create_donut_chart(sch_grade_acci, '2020', '중학교', '사고자학년', '사고수', orders, colors)

In [36]:
# sch_grade_acci 그래프 시각화(2021년, 중등)
create_donut_chart(sch_grade_acci, '2021', '중학교', '사고자학년', '사고수', orders, colors)

In [37]:
# sch_grade_acci 그래프 시각화(2022년, 중등)
create_donut_chart(sch_grade_acci, '2022', '중학교', '사고자학년', '사고수', orders, colors)

In [38]:
# sch_grade_acci 그래프 시각화(2023년, 중등)
create_donut_chart(sch_grade_acci, '2023', '중학교', '사고자학년', '사고수', orders, colors)

##### (3) 고등학교

In [39]:
# sch_grade_acci 그래프 시각화(2019년, 고등)
create_donut_chart(sch_grade_acci, '2019', '고등학교', '사고자학년', '사고수', orders, colors)

In [40]:
# sch_grade_acci 그래프 시각화(2020년, 고등)
create_donut_chart(sch_grade_acci, '2020', '고등학교', '사고자학년', '사고수', orders, colors)

In [41]:
# sch_grade_acci 그래프 시각화(2021년, 고등)
create_donut_chart(sch_grade_acci, '2021', '고등학교', '사고자학년', '사고수', orders, colors)

In [42]:
# sch_grade_acci 그래프 시각화(2022년, 고등)
create_donut_chart(sch_grade_acci, '2022', '고등학교', '사고자학년', '사고수', orders, colors)

In [43]:
# sch_grade_acci 그래프 시각화(2023년, 고등)
create_donut_chart(sch_grade_acci, '2023', '고등학교', '사고자학년', '사고수', orders, colors)

##### (4) 특수학교

In [44]:
# sch_grade_acci 그래프 시각화(2019년, 특수)
create_donut_chart(sch_grade_acci, '2019', '특수학교', '사고자학년', '사고수', orders, colors)

In [45]:
# sch_grade_acci 그래프 시각화(2020년, 특수)
create_donut_chart(sch_grade_acci, '2020', '특수학교', '사고자학년', '사고수', orders, colors)

In [46]:
# sch_grade_acci 그래프 시각화(2021년, 특수)
create_donut_chart(sch_grade_acci, '2021', '특수학교', '사고자학년', '사고수', orders, colors)

In [47]:
# sch_grade_acci 그래프 시각화(2022년, 특수)
create_donut_chart(sch_grade_acci, '2022', '특수학교', '사고자학년', '사고수', orders, colors)

In [48]:
# sch_grade_acci 그래프 시각화(2023년, 특수)
create_donut_chart(sch_grade_acci, '2023', '특수학교', '사고자학년', '사고수', orders, colors)

#### 1-3-5. 학교급별 사고 수 월별 5개년 누적 비교

In [49]:
# sch_month_acci 데이터프레임
sch_month_acci = sch_df.copy()
# 초등학교 저학년/고학년 구분
sch_month_acci.loc[(sch_month_acci['학교급']=='초등학교')&(sch_month_acci['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_month_acci.loc[(sch_month_acci['학교급']=='초등학교')&(sch_month_acci['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 각 사고월별 총 사고 건수 계산
sch_month_tot_acci = sch_month_acci.groupby('사고월')['구분'].count()
# 각 행에 대해 퍼센트를 계산하여 새로운 컬럼 추가
month_level = sch_month_acci.groupby(['학교급','사고월']).count()[['구분']].reset_index()
month_level['퍼센트'] = month_level.apply(lambda row:round((row['구분'] / sch_month_tot_acci[row['사고월']]) * 100, 2), axis=1)
months = ["1월", "2월", "3월", "4월", "5월", "6월", "7월", "8월", "9월", "10월", "11월", "12월"]
month_level_pivot = month_level.pivot_table(index='사고월', columns='학교급', values=['퍼센트','구분']).reindex(index=months)
sch_type_cat = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교", "특수학교", "기타학교"]
colors = ['#5c7dd2','#92b8ff','#aeceff','#c7e4ff','#c3b7eb', '#9590e6', '#837ed5', '#5843a9']

In [56]:
# month_level_pivot 그래프 시각화
create_stacked_barchart(month_level_pivot, sch_type_cat, colors, months)

#### 1-3-6. 학교급별 사고 내용

In [63]:
# 학교급 정렬 기준 지정
custom_order = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교", "특수학교"]
# 시각화를 위한 설정
colors = {
    "유치원": '#5c7dd2', 
    "초등학교(저학년)": '#92b8ff', 
    "초등학교(고학년)": '#aeceff', 
    "중학교": '#c7e4ff', 
    "고등학교": '#c3b7eb', 
    "특수학교": '#9590e6'
    }

##### (1) 사고 시간

In [64]:
# sch_acci_time 데이터프레임(사고시간 5개년 누적)
# '학교급'의 '기타학교' 값 제외
sch_acci_time = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_acci_time.loc[(sch_acci_time['학교급']=='초등학교')&(sch_acci_time['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_acci_time.loc[(sch_acci_time['학교급']=='초등학교')&(sch_acci_time['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_time = sch_acci_time.replace('점심시간', '식사시간')
sch_acci_time = sch_acci_time.replace('석식시간', '식사시간')
sch_acci_time = sch_acci_time.replace('휴식시간 및 청소시간', '휴식/청소시간')
# 연도, 학교급, 사고시간별 사고수 계산
sch_acci_time = sch_acci_time.groupby(['학교급', '사고시간']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
sch_acci_time['학교급'] = pd.Categorical(sch_acci_time['학교급'], categories=custom_order, ordered=True)
sch_acci_time = sch_acci_time.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_time = sch_acci_time.dropna()
sch_acci_time

,학교급,사고시간,사고수
0,유치원,수업시간,22815
1,유치원,기타,4499
2,유치원,식사시간,3299
3,유치원,체육수업,3235
4,유치원,등하교,2329
...,...,...,...
60,특수학교,특별활동,114
61,특수학교,학교행사,98
62,특수학교,체육활동,94
63,특수학교,쉬는시간,92


In [65]:
# sch_acci_time 그래프 시각화
create_sub_barchart(sch_acci_time, '사고시간', '사고수', colors)

##### (2) 사고 장소

In [66]:
# sch_acci_place 데이터프레임(사고장소 5개년 누적)
# '학교급'의 '기타학교' 값 제외
sch_acci_place = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_acci_place.loc[(sch_acci_place['학교급']=='초등학교')&(sch_acci_place['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_acci_place.loc[(sch_acci_place['학교급']=='초등학교')&(sch_acci_place['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_place = sch_acci_place.replace('교외활동', '교외')
# 연도, 학교급, 사고장소별 사고수 계산
sch_acci_place = sch_acci_place.groupby(['학교급', '사고장소']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
sch_acci_place['학교급'] = pd.Categorical(sch_acci_place['학교급'], categories=custom_order, ordered=True)
sch_acci_place = sch_acci_place.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_place = sch_acci_place.dropna()
sch_acci_place

,학교급,사고장소,사고수
0,유치원,교실,21144
1,유치원,부속시설,7756
2,유치원,운동장,7330
3,유치원,통로,3565
4,유치원,교외,2259
5,초등학교(저학년),운동장,20314
6,초등학교(저학년),교실,19511
7,초등학교(저학년),부속시설,19090
8,초등학교(저학년),통로,17563
9,초등학교(저학년),교외,2532


In [68]:
# sch_acci_place 그래프 시각화
create_sub_barchart(sch_acci_place, '사고장소', '사고수', colors)

##### (3) 사고 부위

In [69]:
# sch_acci_part 데이터프레임(사고부위 5개년 누적)
# '학교급'의 '기타학교' 값 제외
sch_acci_part = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_acci_part.loc[(sch_acci_part['학교급']=='초등학교') & (sch_acci_part['사고자학년'].isin(['1학년','2학년','3학년'])), '학교급'] = '초등학교(저학년)'
sch_acci_part.loc[(sch_acci_part['학교급']=='초등학교') & (sch_acci_part['사고자학년'].isin(['4학년','5학년','6학년'])), '학교급'] = '초등학교(고학년)'
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_part['사고부위'] = sch_acci_part['사고부위'].str.replace(r'\([^)]*\)', '', regex=True).str.strip()
# 연도, 학교급, 사고부위별 사고수 계산
sch_acci_part = sch_acci_part.groupby(['학교급', '사고부위']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
sch_acci_part['학교급'] = pd.Categorical(sch_acci_part['학교급'], categories=custom_order, ordered=True)
sch_acci_part = sch_acci_part.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_part = sch_acci_part.dropna()
sch_acci_part

C:\Users\cabby\AppData\Local\Temp\ipykernel_156\313399637.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,학교급,사고부위,사고수
0,유치원,머리,16983
1,유치원,손,5440
2,유치원,치아,5026
3,유치원,발,4502
4,유치원,팔,3313
5,유치원,복합부위,3164
6,유치원,기타,2259
7,유치원,다리,1064
8,유치원,흉복부,303
9,초등학교(저학년),머리,21803


In [70]:
# sch_acci_part 그래프 시각화
create_sub_barchart(sch_acci_part, '사고부위', '사고수', colors)

##### (4) 사고 형태

In [72]:
# sch_acci_type 데이터프레임(사고형태 5개년 누적)
# '학교급'의 '기타학교' 값 제외
sch_acci_type = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_acci_type.loc[(sch_acci_type['학교급']=='초등학교') & (sch_acci_type['사고자학년'].isin(['1학년','2학년','3학년'])), '학교급'] = '초등학교(저학년)'
sch_acci_type.loc[(sch_acci_type['학교급']=='초등학교') & (sch_acci_type['사고자학년'].isin(['4학년','5학년','6학년'])), '학교급'] = '초등학교(고학년)'
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_type = sch_acci_type.replace('낙상-미끄러짐', '낙상')
sch_acci_type = sch_acci_type.replace('낙상-넘어짐', '낙상')
sch_acci_type = sch_acci_type.replace('낙상-떨어짐', '낙상')
sch_acci_type = sch_acci_type.replace('염좌·삐임 등 신체 충격', '신체 충격')
# 연도, 학교급, 사고형태별 사고수 계산
sch_acci_type = sch_acci_type.groupby(['학교급', '사고형태']).size().reset_index(name='사고수')
# 학교급 기준으로 정렬
sch_acci_type['학교급'] = pd.Categorical(sch_acci_type['학교급'], categories=custom_order, ordered=True)
sch_acci_type = sch_acci_type.sort_values(['학교급', '사고수'], ascending=[True, False]).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_type = sch_acci_type.dropna()
sch_acci_type

,학교급,사고형태,사고수
0,유치원,물리적힘 노출,18861
1,유치원,낙상,13476
2,유치원,기타,6315
3,유치원,사람과의 충돌,2759
4,유치원,신체 충격,643
5,초등학교(저학년),낙상,34398
6,초등학교(저학년),물리적힘 노출,29473
7,초등학교(저학년),사람과의 충돌,7375
8,초등학교(저학년),기타,4689
9,초등학교(저학년),신체 충격,3075


In [73]:
# sch_acci_type 그래프 시각화
create_sub_barchart(sch_acci_type, '사고형태', '사고수', colors)

##### (5) 사고 당시 활동

In [74]:
# sch_acci_act 데이터프레임(사고당시활동 5개년 누적)
# '학교급'의 '기타학교' 값 제외
sch_acci_act = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_acci_act.loc[(sch_acci_act['학교급']=='초등학교') & (sch_acci_act['사고자학년'].isin(['1학년','2학년','3학년'])), '학교급'] = '초등학교(저학년)'
sch_acci_act.loc[(sch_acci_act['학교급']=='초등학교') & (sch_acci_act['사고자학년'].isin(['4학년','5학년','6학년'])), '학교급'] = '초등학교(고학년)'
# 연도, 학교급, 사고당시활동별 사고수 계산
sch_acci_act = sch_acci_act.groupby(['학교급', '사고당시활동']).size().reset_index(name='사고수')
# 연도, 학교급 기준으로 정렬
sch_acci_act['학교급'] = pd.Categorical(sch_acci_act['학교급'], categories=custom_order, ordered=True)
sch_acci_act = sch_acci_act.sort_values(['학교급', '사고수']).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_act = sch_acci_act.dropna()
sch_acci_act 

,학교급,사고당시활동,사고수
0,유치원,실험실습,205
1,유치원,구기운동,749
2,유치원,기타운동,2202
3,유치원,식사/수면/휴식,3182
4,유치원,공부,3548
5,유치원,보행/주행,6184
6,유치원,기타,11100
7,유치원,장난/놀이,14884
8,초등학교(저학년),실험실습,936
9,초등학교(저학년),공부,5660


In [75]:
# sch_acci_act 그래프 시각화
create_sub_barchart(sch_acci_act, '사고당시활동', '사고수', colors)

##### (6) 사고 매개물

In [76]:
# sch_acci_mdm 데이터프레임(사고매개물 5개년 누적)
# '학교급'의 '기타학교' 값 제외
sch_acci_mdm = sch_df[sch_df['학교급'] != '기타학교']
# 초등학교 저학년/고학년 구분
sch_acci_mdm.loc[(sch_acci_mdm['학교급']=='초등학교') & (sch_acci_mdm['사고자학년'].isin(['1학년','2학년','3학년'])), '학교급'] = '초등학교(저학년)'
sch_acci_mdm.loc[(sch_acci_mdm['학교급']=='초등학교') & (sch_acci_mdm['사고자학년'].isin(['4학년','5학년','6학년'])), '학교급'] = '초등학교(고학년)'
# 분석 편의를 위해 일부 데이터 값 변경
sch_acci_mdm['사고매개물'] = sch_acci_mdm['사고매개물'].str.replace(r'\([^)]*\)', '', regex=True).str.strip()
# 연도, 학교급, 사고매개물별 사고수 계산
sch_acci_mdm = sch_acci_mdm.groupby(['학교급', '사고매개물']).size().reset_index(name='사고수')
# 연도, 학교급 기준으로 정렬
sch_acci_mdm['학교급'] = pd.Categorical(sch_acci_mdm['학교급'], categories=custom_order, ordered=True)
sch_acci_mdm = sch_acci_mdm.sort_values(['학교급', '사고수']).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_mdm = sch_acci_mdm.dropna()
sch_acci_mdm

C:\Users\cabby\AppData\Local\Temp\ipykernel_156\1218119953.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,학교급,사고매개물,사고수
0,유치원,운송용구,143
1,유치원,기계 도구류,227
2,유치원,열,420
3,유치원,날카로운 물건,773
4,유치원,운동용 장비/기구,5515
5,유치원,건물,7136
6,유치원,자연,7677
7,유치원,가구,9087
8,유치원,기타,11076
9,초등학교(저학년),운송용구,157


In [77]:
# sch_acci_mdm 그래프 시각화
create_sub_barchart(sch_acci_mdm, '사고매개물', '사고수', colors)